## purpose

for this project, I will be wrangling and scraping data from wikipedia that lists neighborhoods and boroughs in Toronto by postal code. I will convert their addresses into their equivalent latitude and longitude values. I will then use the Foursquare API to explore neighborhoods in Downtown Toronto. I will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the _k_-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in Downtown Toronto and their emerging clusters.


In [24]:
#importing necessary libraries 
import pandas as pd
import numpy as np


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

In [13]:
#upload wikipedia table 
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_df = pd.read_html(wiki_url)[0]
toronto_df.rename(columns={"Neighbourhood":'Neighborhood'}) #get rid of Canadian spelling, dontcha know

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [14]:
toronto_df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [15]:
toronto_df_filt = toronto_df[toronto_df['Borough']!='Not assigned'] #dropping Not assigned values 
toronto_df_filt.columns=["Postal Code", "Borough", "Neighborhood"]
toronto_df_filt.reset_index(drop=True, inplace=True)
toronto_df_filt.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
toronto_df_filt.shape

(103, 3)

In [17]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
#combine dfs such that postal codes correspond to locations 
combined_df = pd.merge(toronto_df_filt, df_data_1, on='Postal Code') 

In [19]:
combined_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [20]:
#segmenting only neighborhoods in downtown Toronto 
toronto_data = combined_df[combined_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True) 
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [23]:
#latitude and longitude of toronto 
latitude = 43.6532
longitude = -79.3832

# creating a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [25]:
#utilizing the Foursquare API to explore the neighborhoods and segment them

CLIENT_ID = 'P0JMQZLFNI2ICUI0EDBLJUHSSOBTBZ11L25JVRLLIWB5X23C' # Foursquare ID
CLIENT_SECRET = '0YTNC25I2DD5V2AR0JTK3UTGNSCYI22ENPMKJASY31G5M1WI' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P0JMQZLFNI2ICUI0EDBLJUHSSOBTBZ11L25JVRLLIWB5X23C
CLIENT_SECRET:0YTNC25I2DD5V2AR0JTK3UTGNSCYI22ENPMKJASY31G5M1WI


## create function  to list venues and their category for each neighborhood


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
#run each neighborhood on above function and create new df
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                 latitudes=toronto_data['Latitude'],
                                 longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [37]:
toronto_venues.head()
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
...,...,...,...,...,...,...,...
1220,Church and Wellesley,43.66586,-79.383160,Asahi Sushi,43.669874,-79.382943,Sushi Restaurant
1221,Church and Wellesley,43.66586,-79.383160,A&W,43.666415,-79.378235,Fast Food Restaurant
1222,Church and Wellesley,43.66586,-79.383160,Bishop and Belcher,43.670096,-79.382354,Pub
1223,Church and Wellesley,43.66586,-79.383160,McDonald's,43.668854,-79.385962,Fast Food Restaurant


Analyze each neighborhood by venue category

In [68]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-64]] + list(toronto_onehot.columns[:-1]) #for some reason the 'neighborhood' was in the -64 position, and this moves it the first 
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot = toronto_onehot.loc[:,~toronto_onehot.columns.duplicated()] #must remove the duplicated neighborhood column that is still in position -64

toronto_onehot


,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,Church and Wellesley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1221,Church and Wellesley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1222,Church and Wellesley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1223,Church and Wellesley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [69]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.017241,0.000000,0.000000,0.00,0.00,0.017241,0.00000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.1875,0.0625,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
2,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.015873,0.000000,0.000000,0.00,0.00,0.015873,0.00000,0.000000,0.015873,0.000000
3,Christie,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.012987,0.000000,0.00,...,0.012987,0.012987,0.012987,0.00,0.00,0.000000,0.00000,0.012987,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.040000,0.000000,0.00,...,0.020000,0.000000,0.000000,0.00,0.00,0.020000,0.00000,0.000000,0.010000,0.000000
6,"First Canadian Place, Underground city",0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.030000,0.000000,0.00,...,0.020000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000,0.000000
7,"Garden District, Ryerson",0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.010000,0.020000,0.000000,0.00,0.00,0.000000,0.01000,0.010000,0.010000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.05,...,0.000000,0.010000,0.000000,0.00,0.01,0.010000,0.00000,0.000000,0.010000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.049180,0.00000,0.049180,0.016393,0.000000


In [70]:
#print each neighborhood and top 5 type of venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2              Bakery  0.03
3         Cheese Shop  0.03
4  Seafood Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2           Airport  0.06
3               Bar  0.06
4  Sculpture Garden  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.19
1                Café  0.08
2  Italian Restaurant  0.05
3      Sandwich Place  0.05
4     Bubble Tea Shop  0.03


----Christie----
           venue  freq
0  Grocery Store  0.27
1           Café  0.20
2           Park  0.13
3     Baby Store  0.07
4     Restaurant  0.07


----Church and Wellesley----
                  venue  freq
0           Coffee Shop  0.08
1   Japanese Restaurant  0.06
2      Sushi Restaurant  0.06
3  Fast Food Restaurant  0.04
4   

In [71]:
# function for to sort in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [73]:
##create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Restaurant,Cheese Shop,Jazz Club,Hotel
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Airport Terminal
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Portuguese Restaurant,Poke Place,Middle Eastern Restaurant
3,Christie,Grocery Store,Café,Park,Candy Store,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Baby Store,Dumpling Restaurant
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Fast Food Restaurant,Gay Bar,Restaurant,Pub,Café,Pizza Place,Men's Store


In [74]:
#cluster neighborhoods
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 4], dtype=int32)

In [83]:
#new df that includes clusters
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,0,Airport Service,Airport Lounge,Airport,Bar,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boat or Ferry,Airport Terminal


In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Wine Shop,Electronics Store,Performing Arts Venue
1,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Sushi Restaurant,Bank,Fast Food Restaurant,Smoothie Shop,Beer Bar,Italian Restaurant,Japanese Restaurant,Sandwich Place,Burrito Place
2,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Café,Hotel,Cosmetics Shop,Theater,Diner
3,St. James Town,1,Coffee Shop,Café,Gastropub,American Restaurant,Cocktail Bar,Creperie,Hotel,Beer Bar,Seafood Restaurant,Farmers Market
4,Berczy Park,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Beer Bar,Bakery,Restaurant,Cheese Shop,Jazz Club,Hotel
5,Central Bay Street,1,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Portuguese Restaurant,Poke Place,Middle Eastern Restaurant
7,"Richmond, Adelaide, King",1,Coffee Shop,Café,Restaurant,Clothing Store,Thai Restaurant,Gym,Deli / Bodega,Bakery,Concert Hall,Steakhouse
8,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Aquarium,Café,Hotel,Scenic Lookout,Fried Chicken Joint,Restaurant,Brewery,Italian Restaurant,Sporting Goods Shop
9,"Toronto Dominion Centre, Design Exchange",1,Coffee Shop,Hotel,Café,American Restaurant,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Restaurant,Salad Place,Breakfast Spot
10,"Commerce Court, Victoria Hotel",1,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Deli / Bodega,Italian Restaurant,Japanese Restaurant,Seafood Restaurant


In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Christie,2,Grocery Store,Café,Park,Candy Store,Coffee Shop,Nightclub,Restaurant,Italian Restaurant,Baby Store,Dumpling Restaurant


In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Rosedale,3,Park,Playground,Trail,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,"University of Toronto, Harbord",4,Café,Bar,Bakery,Japanese Restaurant,Bookstore,Italian Restaurant,Beer Bar,Beer Store,Bank,Restaurant
12,"Kensington Market, Chinatown, Grange Park",4,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Bakery,Arts & Crafts Store,Bar,Farmers Market,Gaming Cafe
